###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2022  by D. Koehn, T. Meier and J. Stampa, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Digitale Signalverarbeitung: Korrelation von Signalen

## Kapitel 8 Übungen

### Aufgabe 8.5

Um das Funktionsprinzip von InSAR zu verdeutlichen, sei eine Cosinus-Funktion
mit einer Frequenz von $f_0 = 1\; \text{GHz}$ auf dem Intervall $[0, T ]$, $T = 5\; \text{ps}$ gegeben. Ihre
AKF soll in Abhängigkeit der Änderung des Laufwegs graphisch dargestellt
werden. Hinweis: die Radarwelle breitet sich mit Lichtgeschwindigkeit aus.

Die Idee von InSAR ist die Laufzeitdifferenz zwischen einer abgestrahlten Radarwelle $x(t)$ und der phasenverschobenen Reflexion $y(t)$ von dem Zielobjekt zu bestimmen. Damit können wir das Ergebnis von Aufgabe 8.3 verwenden, um die KKF zwischen $x(t)$ und $y(t)$ zu berechnen. Das Maximum der KKF gibt den Laufzeitunterschied der abgestrahlten Welle und der Reflexion an, woraus sich die Entfernung berechnen läßt.

Die Aufgabenstellung funktioniert so nicht. Bei einer Frequenz von $f_0 = 1\; GHz$ muß eine Zeitfensterlänge von $T = 5\; \text{ns}$ und nicht $T = 5\; \text{ps}$ gewählt werden.

Zur Lösung dieser Aufgabe, können wir den Python-Code aus Aufgabe 8.3 verwenden. Dazu sind folgende Modifikationen erforderlich:

- Entferne die analytischen Lösungen
- Ersetze die Sinus durch die Kosinusfunktion
- Definiere die Länge des Zeitfensters $T = 5\; \text{ns}$ 
- Definiere die Frequenz der Kosinusfunktion $f_0 = 1\; GHz$
- Wähle die maximale Länge der Zeitreihe $tmax = 5*T = 25\; ns$

Was jetzt noch fehlt ist eine Beziehung zwischen Phasenunterschied $\phi$ und Distanz $h$ zwischen Radarquelle und Zielobjekt. Die Laufzeit $t_{se}$ zwischen Quelle, Reflektor und wieder zurück ergibt sich zu:

\begin{equation}
t_{se} = 2 \frac{h}{c} \notag
\end{equation}

mit der Lichtgeschwindigkeit $c$. Die Laufzeit $t_{se}$ führt zu einer Zeitverschiebung der  Sinusfunktion:

\begin{equation}
y(t) = sin(\omega_0 (t-t_{se})) \notag
\end{equation}

In [2]:
# Importiere Python Bibliotheken 
# ------------------------------
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy import constants   # import constants including the speed of light c [m/s]

In [3]:
def KKF_xy(h):
    
    # Define parameters
    T = 5e-9                        # time window T = 5 ns
    f0 = 1e9                        # f0 = 1 GHz
    tmax = 5.* T                    # set maximum time tmax to 5*T
    dt = tmax / 4000                # sampling interval [s] is 1/4000 of tmax
    
    # Define time vector
    nt = (int) (tmax/dt)            # number of time samples 
    time = np.arange(0,nt*dt,dt)    # define time vector
    
    # Define cos function x(t) on time interval T
    # -------------------------------------------
    w = 2 * np.pi * f0           # circular frequency
    x = np.cos(w*time)           # calculate cos function

    # set cos function outside [0,T] to zero
    ntmax = (int)(T/dt)
    x[ntmax:] = 0.0
    
    # Define cos function y(t) with time shift tse on time interval [tse,T+tse]
    # -------------------------------------------------------------------------
    tse = 2 * h / constants.c   # compute 2*traveltime between source and target       
    y = np.cos(w*(time-tse))    # calculate cos function

    # set cos function y(t) outside [tse,T+tse] to zero
    ntse = (int)(tse/dt)
    y[:ntse] = 0.0
    y[ntmax+ntse:] = 0.0
    
    # KKF(x,y)
    KKF_xy = signal.correlate(y, x, mode='same')

    # Multiply KKF_xy with dt and divide by T to get correct amplitudes
    KKF_xy = KKF_xy * dt / T

    # Compute time-lags
    nt = len(KKF_xy)                    # number of samples
    lags = np.arange(-nt/2,nt/2,1) * dt # time-lags
    
    # Estimate distance between source and target using maximum of KKF
    ntpick = np.argmax(KKF_xy)  # Estimate time sample where maximum of KKF
    lagpick = lags[ntpick]      # Estimate time-lag between source and target
    
    # Estimate distance hKKF between source and target from lagpick
    hKKF = constants.c * lagpick / 2. 
    
    # Plot time series x(t), y(t) and their KKF_xy
    plt.figure(figsize=(20,10))
    
    plt.subplot(211)
    
    plt.plot(time/1e-9, x, 'b-',lw=3,label='x(t)')
    plt.plot(time/1e-9, y, 'r-',lw=3,label='y(t)')
    plt.title('x(t), y(t)')
    plt.xlabel('time [ns]')
    plt.ylabel('Cosine function x(t), y(t)')
    plt.legend()
    plt.grid()
    
    hKKF_str = 'hKKF = ' + str(np.round(hKKF,decimals=4)) + ' m'
    plt.text(18, 0.7, hKKF_str, size=30, rotation=0.,
    ha="center", va="center",
    bbox=dict(boxstyle="round",
    ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8),))
    
    plt.subplot(212)
    
    # plot KKF
    plt.plot(lags/1e-9, KKF_xy, 'b-',lw=3)
    plt.plot(lagpick/1e-9, KKF_xy[ntpick], 'ro',lw=3)
    
    plt.title('KKF(x,y)')
    plt.xlabel(r'time-lags $\tau$ [ns]')
    plt.ylabel('KKF(x,y)')
    plt.grid()

Eingabeparameter der Funktion `KKF_xy` ist die Entfernung $h\in[1\;mm,1\;m]$ zwischen Radarquelle und Zielobjekt.

In [4]:
interactive_plot = interactive(KKF_xy, h=(0.001, 1., 0.001))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='h', max=1.0, min=0.001, step=0.001), Output(layout=L…

Der Time-Lag des Maximums der KKF entspricht der Laufzeitdifferenz zwischen Radarquelle und Zielobjekt. Aus dieser läßt sich bei bekannter Lichtgeschwindigkeit die Entfernung $\text{hKKF}$ zwischen Quelle und Reflektor bestimmen.